In [17]:
import os
os.environ['KERAS_BACKEND'] = 'torch'

In [64]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Concatenate

moves_input = Input((None, 1), name='moves_input')
lstm_layer = LSTM(16, return_sequences=True)(moves_input)
lstm_out = LSTM(32, return_sequences=False)(lstm_layer)

metadata_input = Input((7,), name='metadata_input')
metadata_hidden = Dense(16, name='metadata_hidden', activation='relu')(metadata_input)

combined = Concatenate()([lstm_layer, metadata_hidden])
combined_dense = Dense(80, activation='relu')(combined)

white_elo_output = Dense(1, name="white_elo_output")(combined_dense)
black_elo_output = Dense(1, name="black_elo_output")(combined_dense)

model = Model(inputs=[moves_input, metadata_input], outputs=[white_elo_output, black_elo_output])

In [65]:
import keras.metrics

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[
    keras.metrics.RootMeanSquaredError(name="rmse", dtype=None),
    keras.metrics.R2Score(class_aggregation="uniform_average", name="r2")
])

In [33]:
DATASET = 'data/classical_1dim.h5'
import pandas as pd


df = pd.read_hdf(DATASET)

import numpy as np

df[["result_0-1", "result_1-0", "result_1/2-1/2",
    "termination_normal", "termination_rules infraction", "termination_time forfeit"]] = \
df[["result_0-1", "result_1-0", "result_1/2-1/2",
    "termination_normal", "termination_rules infraction", "termination_time forfeit"]].astype(np.float32)

# Features (X)
X_moves = df['moves'].values
X_metadata = df[["time", "increment", "result_0-1", "result_1-0", "result_1/2-1/2",
                 "termination_normal", "termination_rules infraction"]].values

# Targets (y)
y_white_elo = df["white_elo"].values
y_black_elo = df["black_elo"].values

In [67]:
from sklearn.model_selection import train_test_split

X_moves_train, X_moves_test, X_metadata_train, X_metadata_test, y_white_train, y_white_test, y_black_train, y_black_test = train_test_split(
    X_moves, X_metadata, y_white_elo, y_black_elo, test_size=0.75, random_state=42
)

In [68]:
X_moves_train

array([array([-0.95061314, -0.9600917 , -0.9883789 , -0.95966834, -0.96980244,
              -0.95958155, -0.9825089 , -0.973848  , -0.9802083 , -0.9891493 ,
              -0.9822517 , -0.97700936, -0.9825445 , -0.9847624 , -0.958853  ,
              -0.9539115 , -0.99399835, -0.9748069 , -0.9690236 , -0.97935003,
              -0.99176013, -0.98310924, -0.99259543, -0.99259543, -1.0034198 ,
              -1.0039248 , -0.9951246 , -0.9907356 , -0.97142196, -1.010162  ,
              -0.991901  , -0.97545   , -0.9999105 , -0.99878055, -0.98034745,
              -0.9887577 , -0.98805887, -0.98805887, -0.95874673, -0.97888803,
              -0.97554153, -0.97554153, -0.9960659 , -0.97748893, -0.9775624 ,
              -1.0004493 , -0.9831635 , -0.97023314, -0.9951033 , -0.98010695,
              -0.9994664 , -0.9907967 , -0.9747259 , -0.96532845, -0.96768576,
              -0.9576553 , -0.97528225, -0.94598347, -0.9758766 , -0.98670805,
              -0.999612  , -0.9879972 , -0.982459  ,

In [69]:
from keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(X_moves_train, padding='post', dtype='float32')
padded = np.array(padded)
padded.shape

(61623, 321)

In [70]:
tb_callback = keras.callbacks.TensorBoard('./logs', update_freq=1)

model.fit(
    [padded, X_metadata], 
    [y_white_train, y_black_train], 
    epochs=1, 
    batch_size=128,
    validation_split=0.2, 
    verbose=1
)

209/386 ━━━━━━━━━━━━━━━━━━━━ 2:46 939ms/step - black_elo_output_loss: 0.0193 - black_elo_output_r2: -0.0064 - loss: 0.0355 - white_elo_output_loss: 0.0163 - white_elo_output_rmse: 0.1275

KeyboardInterrupt: 